# Distribution by ownership type

In [1]:
import fiona
import numpy as np
import rasterio
from rasterio import features
from shapely.geometry import shape
from tqdm import tqdm
import os
from rasterio.windows import Window, bounds, from_bounds
from rasterio.enums import Resampling
import pandas as pd

# Table of contents

#### [1. Base layer](#Base-layer)

#### [2. Rasterizing cadastral map by ownership type](#Rasterizing-cadastral-map-by-ownership-type)

#### [3. Tiff multiplication](#Tiff-multiplication)

#### [4. Pixel count function](#Pixel-count-function)

#### [5. Result](#Result)



## Base layer
[[Click here to turn back to the table of contents]](#Table-of-contents)

In [2]:
#taking parameters from Estonian layer
path_scenario_1_r = '../../Thesis_2024_updates/Raster_Data/Scenario_1/'
file_name_whole_estonia_r = 'whole_estonia/Estonia_rasterized_0_1.tif'
raster_base_layer = os.path.join(path_scenario_1_r, file_name_whole_estonia_r)

with rasterio.open(raster_base_layer) as src:
    base_trans = src.transform
    base_profile = src.profile
    base_shape = src.shape
    base_data = src.read(1)
    
# Create a boolean mask where True represents pixels with value 1
mask = base_data == 1

# Count the number of True values in the mask
num_total_pixels = np.sum(mask)
total_area_ha = round((num_total_pixels * 16)/10000)

print("Number of pixels equal to 1:", num_total_pixels)
print("Total area of Estonia: {} ha".format(total_area_ha))

Number of pixels equal to 1: 2716698675
Total area of Estonia: 4346718 ha


## Rasterizing cadastral map by ownership type
[[Click here to turn back to the table of contents]](#Table-of-contents)

Rasterization was done in QGIS.   
Firstly, the ownership type data was translated to integer numbers using the following rules:

CASE  
WHEN  "OMVORM"  = 'Eraomand' THEN 1  
WHEN  "OMVORM"  =  'Riigiomand' THEN 2  
WHEN  "OMVORM"  =  'Munitsipaalomand' THEN 3  
WHEN  "OMVORM"  =  'Omandi ulatus selgitamisel' THEN 4  
WHEN  "OMVORM"  =  'Segaomand' THEN 5  
WHEN  "OMVORM"  =  'Kinnistamata eraomand' THEN 6  
WHEN  "OMVORM"  =  'Avalik-õiguslik omand' THEN 7  
ELSE 8  
END  


After that, the "Rasterize (vector to raster)" function was applied

In [3]:
ownership_mapping = {
    1: 'Private Property',
    2: 'State Property',
    3: 'Municipal Property',
    4: 'Ownership extent under clarification',
    5: 'Mixed Ownership',
    6: 'Unsecured Private Property',
    7: 'Public Property',
    8: 'Other values',
    0: 'NoData'
}

# Tiff multiplication
[[Click here to turn back to the table of contents]](#Table-of-contents)

In [4]:
def multiply_tif_from_base(tif_base, tif_to_multiply, tif_output, tile_size=16384, need_to_resample=False):
    
    # Open the base raster for multiplication and read its profile and nodata value
    with rasterio.open(tif_base) as src:
        profile = src.profile
        nodata1 = src.meta['nodata']
    
    # Open the raster to be multiplied and read its nodata value
    with rasterio.open(tif_to_multiply) as src:
        nodata2 = src.meta['nodata']
        
    # Open both raster datasets for reading data
    tif_base_src = rasterio.open(tif_base)
    tif_to_multiply_src = rasterio.open(tif_to_multiply)
    
    # Open a new raster file for writing the result of the multiplication
    with rasterio.open(tif_output, 'w', **profile) as target_src:
        with tqdm(total=tif_base_src.height * tif_base_src.width // tile_size // tile_size) as pbar:
            for y in range(0, tif_base_src.height, tile_size):
                for x in range(0, tif_base_src.width, tile_size):
                    w1 = Window(x, y, min(tile_size, tif_base_src.width - x), min(tile_size, tif_base_src.height - y))
                    img1 = tif_base_src.read(1, window=w1)  # Read band 1

                    # Replace 'no data' with 0 for img1
                    if nodata1 is not None:
                        img1[img1 == nodata1] = 0

                    # Calculate the bounds of the window and adjust it for the multiply raster
                    bound = bounds(w1, tif_base_src.transform)
                    w2 = from_bounds(*bound, transform=tif_to_multiply_src.transform, width=w1.width, height=w1.height)
                    
                    img2 = tif_to_multiply_src.read(1, window=w2, boundless=True, resampling=Resampling.nearest if need_to_resample else Resampling.nearest)

                    # Replace 'no data' with 0 for img2
                    if nodata2 is not None:
                        img2[img2 == nodata2] = 0

                    # Perform multiplication
                    res = img1.astype(np.float32) * img2.astype(np.float32)
                    target_src.write(res.astype(np.uint8), 1, window=w1)  # Write to band 1
                    pbar.update()



# Pixel count function
[[Click here to turn back to the table of contents]](#Table-of-contents)

In [5]:
def count_pixels_by_type_windowed_to_df(geotiff_file, window_size=16384):
    pixel_counts = {}

    # Open the GeoTIFF file
    with rasterio.open(geotiff_file) as src:
        # Determine the number of windows in each dimension
        for j in range(0, src.height, window_size):
            for i in range(0, src.width, window_size):
                # Define the window
                window = Window(i, j, min(window_size, src.width - i), min(window_size, src.height - j))
                # Read data in the window
                data = src.read(1, window=window)

                # Update counts for each unique value in this chunk
                unique, counts = np.unique(data, return_counts=True)
                for val, count in zip(unique, counts):
                    if val in pixel_counts:
                        pixel_counts[val] += count
                    else:
                        pixel_counts[val] = count

    # Convert the dictionary to a DataFrame
    pixel_df = pd.DataFrame(list(pixel_counts.items()), columns=['Pixel Value', 'Count'])

    return pixel_df


# Result
[[Click here to turn back to the table of contents]](#Result)

In [6]:
# Paths to the files
tif_base = '../../Thesis_2024_updates/Subtraction_Results/Final_file.tif'
tif_to_multiply = '../../Thesis_2024_updates/Raster_Data/cadastral_units/cadastral_units_rasterized.tif'
tif_output = '../../Thesis_2024_updates/Raster_Data/result/distribution_ownership.tif'

# Multiply layers
multiply_tif_from_base(tif_base, tif_to_multiply, tif_output)


  0%|                                                                                                                                                                                | 0/22 [00:00<?, ?it/s]/Users/rav/dev/virtual_env/lib/python3.9/site-packages/rasterio/windows.py:314: RasterioDeprecationWarning: The height, width, and precision parameters are unused, deprecated, and will be removed in 2.0.0.
  warnings.warn(
24it [06:01, 15.06s/it]                                                                                                                                                                                     


In [8]:
# Count pixels by ownership type 
working_tif = '../../Thesis_2024_updates/Raster_Data/result/distribution_ownership.tif'
pixel_df = count_pixels_by_type_windowed_to_df(working_tif, window_size=16384)

# Calculate the area in hectares
pixel_df['area_ha'] = pixel_df['Count'].apply(lambda x: round((x * 16)/10000))

# Apply the mapping
pixel_df['Translated Pixel Value'] = pixel_df['Pixel Value'].map(ownership_mapping)

# Calculate % of total potentially suitable area
total_suitable_area = 130160
pixel_df['%'] = pixel_df['area_ha'].apply(lambda x: round((x/total_suitable_area) * 100, 1))

pixel_df[pixel_df['Pixel Value'] != 0]

,Pixel Value,Count,area_ha,Translated Pixel Value,%
1,1,64144222,102631,Private Property,78.8
2,2,10549302,16879,State Property,13.0
3,3,5670209,9072,Municipal Property,7.0
4,4,456225,730,Ownership extent under clarification,0.6
5,5,75124,120,Mixed Ownership,0.1
6,6,11442,18,Unsecured Private Property,0.0
7,7,83806,134,Public Property,0.1
8,8,15,0,Other values,0.0
